In [1]:
import numpy as np
import pyvista as pv

# 网格尺寸
width, height = 3600, 2400
n_points = width * height

# 路径配置
filename_u = "uf.dat"
filename_v = "vf.dat"
output_filename = "uv_mesh.vtp"

# ================================
# Step 1: 读取 .raw 向量场数据
# ================================

# 读取 float32 原始数据
raw_u = np.fromfile(filename_u, dtype=np.float32)
raw_v = np.fromfile(filename_v, dtype=np.float32)

# 方式一：分段存储：前一半是 u，后一半是 v
u = raw_u.reshape((height, width))
v = raw_v.reshape((height, width))


# ================================
# Step 2: 构造顶点坐标 & uv 向量
# ================================
x = np.arange(width)
y = np.arange(height)
xx, yy = np.meshgrid(x, y)

# 顶点坐标 [x, y, 0]
points = np.column_stack((xx.ravel(), yy.ravel(), np.zeros_like(xx).ravel()))

# uv 向量场作为 point data
uv = np.stack((u.ravel(), v.ravel()), axis=1)  # shape (N, 2)

# ================================
# Step 3: 构造三角形单元
# ================================
cells = []
for i in range(height - 1):
    for j in range(width - 1):
        v0 = i * width + j
        v1 = v0 + 1
        v2 = v0 + width
        v3 = v2 + 1
        # 注意 VTK 需要前缀数量
        cells.extend([3, v0, v1, v3])  # 上三角
        cells.extend([3, v0, v3, v2])  # 下三角

cells = np.array(cells)

# ================================
# Step 4: 构造 PyVista 网格
# ================================
mesh = pv.PolyData()
mesh.points = points
mesh.faces = cells
mesh["uv"] = uv  # 附加向量场

# 保存为 .vtp 文件
mesh.save(output_filename)
print(f"UV vector mesh saved to {output_filename}")

# ================================
# Step 5: 示例：根据 cell id 查询 uv
# ================================
def get_uv_by_cellid(cell_id):
    cell = mesh.get_cell(cell_id)
    uv_values = mesh["uv"][cell.point_ids]
    return uv_values  # shape (3, 2)

# 示例：获取第 12345 个三角形的 uv 向量
example_uv = get_uv_by_cellid(12345)
print("UV vectors for triangle 12345:\n", example_uv)


KeyboardInterrupt: 

In [ ]:
example_uv = get_uv_by_cellid(12344)
print("UV vectors for triangle 12345:\n", example_uv)
example_uv = get_uv_by_cellid(12345)
print("UV vectors for triangle 12345:\n", example_uv)

#  2,=========1,1
#   =       = 
#   =    =  =    
#   =  =    =
# 3,2========= ,3


UV vectors for triangle 12345:
 [[0. 0.]
 [0. 0.]
 [0. 0.]]
UV vectors for triangle 12345:
 [[0. 0.]
 [0. 0.]
 [0. 0.]]


In [11]:
import pyvista as pv
import numpy as np
height, width = 2400, 3600
# 读取 .vtp 文件
mesh_ori = pv.read("uv_mesh.vtp")
mesh_ori_u = np.fromfile("uf.dat", dtype=np.float32).reshape((height, width))
mesh_ori_v = np.fromfile("vf.dat", dtype=np.float32).reshape((height, width))

# # 查看信息
# print(mesh)
# print("Number of points:", mesh.n_points)
# print("Number of cells:", mesh.n_cells)
# print("Available point data arrays:", mesh.point_data.keys())
# print("Available cell data arrays:", mesh.cell_data.keys())
mesh_sz3 = pv.read("uv_mesh_sz3.vtp")
mesh_sz3_u = np.fromfile("uf.sz3.out", dtype=np.float32).reshape((height, width))
mesh_sz3_v = np.fromfile("vf.sz3.out", dtype=np.float32).reshape((height, width))

In [33]:
def find_true_critical_cells(u, v, epsilon=1e-6, residual_tol=1e-6):
    """
    输入:
        u, v: shape=(H, W) 的 2D 数组，表示向量场
    输出:
        critical_cell_ids: 满足条件的三角形 ID（0-based），总数为 (H-1)*(W-1)*2 个
    """
    H, W = u.shape
    n_cells = (H - 1) * (W - 1) * 2

    # 构造所有三角形的三组向量
    def build_triangle_uvs(u, v):
        # 点索引（左上角）
        v0 = np.stack([u[:-1, :-1], v[:-1, :-1]], axis=-1).reshape(-1, 2)
        v1 = np.stack([u[:-1, 1:],  v[:-1, 1:]],  axis=-1).reshape(-1, 2)
        v2 = np.stack([u[1:, :-1], v[1:, :-1]],  axis=-1).reshape(-1, 2)
        v3 = np.stack([u[1:, 1:],  v[1:, 1:]],   axis=-1).reshape(-1, 2)

        # 上三角：v0, v1, v3
        tri1 = np.stack([v0, v1, v3], axis=1)  # shape (N, 3, 2)
        # 下三角：v0, v3, v2
        tri2 = np.stack([v0, v3, v2], axis=1)

        return np.concatenate([tri1, tri2], axis=0)

    all_uvs = build_triangle_uvs(u, v)  # shape (n_cells, 3, 2)
    v0, v1, v2 = all_uvs[:, 0], all_uvs[:, 1], all_uvs[:, 2]

    # # 过滤掉任意一个为 0 的
    # zero_mask = (
    #     (np.linalg.norm(v0, axis=1) == 0.0) |
    #     (np.linalg.norm(v1, axis=1) == 0.0) |
    #     (np.linalg.norm(v2, axis=1) == 0.0)
    # )

    # v0, v1, v2 = v0[~zero_mask], v1[~zero_mask], v2[~zero_mask]
    # valid_ids = np.where(~zero_mask)[0]

    A = np.stack([v0 - v2, v1 - v2], axis=2)
    b = -v2

    # 判定矩阵是否可逆
    det = A[:, 0, 0]*A[:, 1, 1] - A[:, 0, 1]*A[:, 1, 0]
    nonsingular = np.abs(det) > 0
    A, b = A[nonsingular], b[nonsingular]
    v0, v1, v2 = v0[nonsingular], v1[nonsingular], v2[nonsingular]
    valid_ids = np.where(nonsingular)[0]

    λ = np.linalg.solve(A, b)
    λ0, λ1 = λ[:, 0], λ[:, 1]
    λ2 = 1 - λ0 - λ1

    inside = (
        (λ0 >= 0) & (λ0 <= 1) &
        (λ1 >= 0) & (λ1 <= 1) &
        (λ2 >= 0) & (λ2 <= 1)
    )

    # v_interp = λ0[:, None] * v0 + λ1[:, None] * v1 + λ2[:, None] * v2
    # residual = np.linalg.norm(v_interp, axis=1)
    # close_to_zero = residual < residual_tol

    # final_mask = inside & close_to_zero
    final_mask = inside 
    return valid_ids[final_mask]  # 返回所有 true critical cell 的 flat ID

critical_cells = find_true_critical_cells(mesh_ori_u, mesh_ori_v)
print(f"检测到 {len(critical_cells)} 个包含 critical point 的三角形")

检测到 90826 个包含 critical point 的三角形


In [34]:
critical_cells = find_true_critical_cells(mesh_ori_u, mesh_ori_v)
print(f"检测到 {len(critical_cells)} 个包含 critical point 的三角形 in original mesh")
critical_cells_sz3 = find_true_critical_cells(mesh_sz3_u, mesh_sz3_v)
print(f"检测到 {len(critical_cells_sz3)} 个包含 critical point 的三角形 in sz3 mesh")

检测到 90826 个包含 critical point 的三角形 in original mesh
检测到 94374 个包含 critical point 的三角形 in sz3 mesh


In [35]:
def build_critical_bitmap(u, v):
    """
    输入:
        u, v: shape=(H, W) 的 2D numpy 数组，表示向量场
    输出:
        bitmap: shape=(H-1, W-1, 2)，每个格子中两个三角形是否含 critical point
    """
    H, W = u.shape
    n_cells = (H - 1) * (W - 1) * 2

    # 获取所有的 critical triangle ids
    critical_ids = find_true_critical_cells(u, v)

    # 初始化 bitmap
    bitmap = np.zeros((H - 1, W - 1, 2), dtype=np.uint8)

    for cid in critical_ids:
        grid_cell_id = cid // 2
        tri_idx = cid % 2  # 0: upper, 1: lower
        row = grid_cell_id // (W - 1)
        col = grid_cell_id % (W - 1)
        bitmap[row, col, tri_idx] = 1

    return bitmap

bitmap_ori = build_critical_bitmap(mesh_ori_u, mesh_ori_v)
print("bitmap_ori shape:", bitmap_ori.shape)  # (2399, 3599, 2)
print("CP 数量总计:", np.sum(bitmap_ori))


bitmap_sz3 = build_critical_bitmap(mesh_sz3_u, mesh_sz3_v)
print("bitmap_sz3 shape:", bitmap_sz3.shape)  # (2399, 3599, 2)
print("CP 数量总计:", np.sum(bitmap_sz3))
bitmap_sz3_hard_copy = bitmap_sz3.copy()

bitmap_ori shape: (2399, 3599, 2)
CP 数量总计: 90826
bitmap_sz3 shape: (2399, 3599, 2)
CP 数量总计: 94374


In [47]:
import numpy as np
import cvxpy as cp

# Compute lambda from u, v (same as your original logic)
def compute_lambda_expr(u_vars, v_vars):
    du = u_vars[0] - u_vars[1]
    dv = v_vars[0] - v_vars[1]
    x = (2 * du - dv) / 3
    y = (-du + 2 * dv) / 3
    return cp.hstack([1 - x - y, x, y])

def get_var(ii, jj, patch_u, patch_v, u_dec, v_dec, constraints, objective_terms, epsilon):
    if (ii, jj) not in patch_u:
        patch_u[(ii, jj)] = cp.Variable()
        patch_v[(ii, jj)] = cp.Variable()
        # Bound within epsilon from decompressed value
        constraints += [
            patch_u[(ii, jj)] >= u_dec[ii, jj] - epsilon,
            patch_u[(ii, jj)] <= u_dec[ii, jj] + epsilon,
            patch_v[(ii, jj)] >= v_dec[ii, jj] - epsilon,
            patch_v[(ii, jj)] <= v_dec[ii, jj] + epsilon
        ]
        # Objective: stay close to decompressed data
        objective_terms.append(cp.abs(patch_u[(ii, jj)] - u_dec[ii, jj]))
        objective_terms.append(cp.abs(patch_v[(ii, jj)] - v_dec[ii, jj]))
    return patch_u[(ii, jj)], patch_v[(ii, jj)]

def patchwise_milp_fix(u_dec, v_dec, bitmap_ori, bitmap_dec, patch_size=50, epsilon=0.05, tau=1e-4, M=10.0):
    H, W = u_dec.shape  # (3600, 2400)
    u_fixed = u_dec.copy()
    v_fixed = v_dec.copy()

    total_patches = 0
    total_triangles_fixed = 0

    # Iterate over patches
    for i in range(0, H - 1, patch_size):
        for j in range(0, W - 1, patch_size):
            print(f"Processing patch i={i}, j={j}")

            patch_u = {}
            patch_v = {}
            constraints = []
            objective_terms = []
            triangle_count = 0

            i_end = min(i + patch_size, H - 1)
            j_end = min(j + patch_size, W - 1)

            # Loop through each cell in this patch
            for ii in range(i, i_end):
                for jj in range(j, j_end):
                    for tri_id in range(2):
                        if not (bitmap_ori[ii, jj, tri_id] == 1 and bitmap_dec[ii, jj, tri_id] == 0):
                            continue

                        # Get triangle vertices
                        if tri_id == 0:  # lower triangle (↙)
                            vtx = [(ii, jj), (ii + 1, jj), (ii + 1, jj + 1)]
                        else:           # upper triangle (↗)
                            vtx = [(ii, jj), (ii, jj + 1), (ii + 1, jj + 1)]

                        u_vars = [get_var(*v, patch_u, patch_v, u_dec, v_dec, constraints, objective_terms, epsilon)[0] for v in vtx]
                        v_vars = [get_var(*v, patch_u, patch_v, u_dec, v_dec, constraints, objective_terms, epsilon)[1] for v in vtx]
                        lambda_vec = compute_lambda_expr(u_vars, v_vars)

                        # Add binary variables for lambda out-of-bound
                        z_low = [cp.Variable(boolean=True) for _ in range(3)]
                        z_high = [cp.Variable(boolean=True) for _ in range(3)]

                        for k in range(3):
                            constraints += [
                                lambda_vec[k] <= -tau + M * (1 - z_low[k]),
                                lambda_vec[k] >= 1 + tau - M * (1 - z_high[k]),
                                z_low[k] + z_high[k] <= 1
                            ]

                        # At least one lambda must be out of bounds
                        constraints.append(cp.sum(z_low + z_high) >= 1)
                        triangle_count += 1

            if not constraints:
                continue  # nothing to fix in this patch

            total_patches += 1
            total_triangles_fixed += triangle_count
            print(f"  -> Fixing {triangle_count} triangles in patch ({i},{j})")

            # Solve MILP for this patch
            prob = cp.Problem(cp.Minimize(cp.sum(objective_terms)), constraints)
            try:
                prob.solve(solver=cp.GUROBI, verbose=False, reoptimize=True)
            except Exception as e:
                print(f"  [Error] Solver failed in patch ({i},{j}): {str(e)}")
                continue

            if prob.status != "optimal":
                print(f"  [Warning] MILP not solved optimally at patch ({i},{j})")
                continue

            # Write back result
            for (ii, jj), var in patch_u.items():
                u_fixed[ii, jj] = var.value
            for (ii, jj), var in patch_v.items():
                v_fixed[ii, jj] = var.value

    print(f"\n[Summary] Total patches processed: {total_patches}")
    print(f"[Summary] Total triangles fixed: {total_triangles_fixed}\n")

    return u_fixed, v_fixed


In [35]:
u_fixed, v_fixed = patchwise_milp_fix(mesh_sz3_u, mesh_sz3_v, bitmap_ori, bitmap_sz3, patch_size=500, epsilon=0.1, tau=0, M=10.0)

Processing patch i=0, j=0
  -> Fixing 103 triangles in patch (0,0)
  [Warning] MILP not solved optimally at patch (0,0)
Processing patch i=0, j=500
  -> Fixing 127 triangles in patch (0,500)
  [Warning] MILP not solved optimally at patch (0,500)
Processing patch i=0, j=1000
  -> Fixing 52 triangles in patch (0,1000)
Processing patch i=0, j=1500
  -> Fixing 50 triangles in patch (0,1500)
Processing patch i=0, j=2000
  -> Fixing 46 triangles in patch (0,2000)
Processing patch i=0, j=2500
  -> Fixing 89 triangles in patch (0,2500)
Processing patch i=0, j=3000
  -> Fixing 86 triangles in patch (0,3000)
  [Warning] MILP not solved optimally at patch (0,3000)
Processing patch i=0, j=3500
  -> Fixing 9 triangles in patch (0,3500)
Processing patch i=500, j=0
  -> Fixing 70 triangles in patch (500,0)
Processing patch i=500, j=500
  -> Fixing 68 triangles in patch (500,500)
Processing patch i=500, j=1000
  -> Fixing 55 triangles in patch (500,1000)
  [Warning] MILP not solved optimally at patch 

KeyboardInterrupt: 

In [36]:
# 尝试fallback：如果一个 patch 内的 MILP 无法最优求解（非 optimal），会自动 fallback 成 单个三角形逐个优化；
import numpy as np
import cvxpy as cp

# def triangle_has_zero_vertex(u_dec, v_dec, vtx, threshold=1e-6):
def triangle_has_zero_vertex(u_dec, v_dec, vtx, threshold=0):
    for ii, jj in vtx:
        if np.hypot(u_dec[ii, jj], v_dec[ii, jj]) < threshold:
            return True
    return False
# Compute lambda from u, v 
def compute_lambda_expr(u_vars, v_vars):
    du = u_vars[0] - u_vars[1]
    dv = v_vars[0] - v_vars[1]
    x = (2 * du - dv) / 3
    y = (-du + 2 * dv) / 3
    return cp.hstack([1 - x - y, x, y])

def compute_lambda_numpy(u_vals, v_vals):
    A = np.array([[2, 1], [1, 2]])
    b = np.array([u_vals[0] - u_vals[1], v_vals[0] - v_vals[1]])
    x, y = np.linalg.solve(A, b)
    return np.array([1 - x - y, x, y])

# def is_outside(lambda_vec, tau=1e-3):
#     return np.any(lambda_vec < -tau) or np.any(lambda_vec > 1 + tau)
def is_outside(lambda_vec):
    return np.any(lambda_vec < 0.0) or np.any(lambda_vec > 1.0)

def get_var(ii, jj, patch_u, patch_v, u_dec, v_dec, constraints, objective_terms, epsilon):
    if (ii, jj) not in patch_u:
        patch_u[(ii, jj)] = cp.Variable()
        patch_v[(ii, jj)] = cp.Variable()
        # Bound within epsilon from decompressed value
        constraints += [
            patch_u[(ii, jj)] >= u_dec[ii, jj] - epsilon,
            patch_u[(ii, jj)] <= u_dec[ii, jj] + epsilon,
            patch_v[(ii, jj)] >= v_dec[ii, jj] - epsilon,
            patch_v[(ii, jj)] <= v_dec[ii, jj] + epsilon
        ]
        # Objective: stay close to decompressed data
        objective_terms.append(cp.abs(patch_u[(ii, jj)] - u_dec[ii, jj]))
        objective_terms.append(cp.abs(patch_v[(ii, jj)] - v_dec[ii, jj]))
    return patch_u[(ii, jj)], patch_v[(ii, jj)]

def fallback_single_triangle(u_dec, v_dec, u_fixed, v_fixed, vtx, epsilon, tau=1e-4, M=10.0):
    if triangle_has_zero_vertex(u_dec, v_dec, vtx):
        # print(f"[Skip] Triangle {vtx} skipped due to zero vertex")
        return False
    
    u_vals = np.array([u_dec[ii, jj] for (ii, jj) in vtx])
    v_vals = np.array([v_dec[ii, jj] for (ii, jj) in vtx])

    A = np.array([[2, 1], [1, 2]])
    b = np.array([u_vals[0] - u_vals[1], v_vals[0] - v_vals[1]])
    x, y = np.linalg.solve(A, b)
    lambda_dec = np.array([1 - x - y, x, y])
    
    u_vars = [cp.Variable() for _ in range(3)]
    v_vars = [cp.Variable() for _ in range(3)]
    constraints = []
    objective = 0
    for i in range(3):
        ii, jj = vtx[i]
        constraints += [
            u_vars[i] >= u_dec[ii, jj] - epsilon,
            u_vars[i] <= u_dec[ii, jj] + epsilon,
            v_vars[i] >= v_dec[ii, jj] - epsilon,
            v_vars[i] <= v_dec[ii, jj] + epsilon
        ]
        objective += cp.abs(u_vars[i] - u_dec[ii, jj]) + cp.abs(v_vars[i] - v_dec[ii, jj])

    lambda_vec = compute_lambda_expr(u_vars, v_vars)
    z_low = [cp.Variable(boolean=True) for _ in range(3)]
    z_high = [cp.Variable(boolean=True) for _ in range(3)]
    for k in range(3):
        constraints += [
            # lambda_vec[k] <= -tau + M * (1 - z_low[k]),
            # lambda_vec[k] >= 1 + tau - M * (1 - z_high[k]),
            lambda_vec[k] <= -0.001 + M * (1 - z_low[k]), #tau not too small
            lambda_vec[k] >= 1 +0.001 - M * (1 - z_high[k]),
            z_low[k] + z_high[k] <= 1
        ]
    constraints.append(cp.sum(z_low + z_high) >= 1)

    prob = cp.Problem(cp.Minimize(objective), constraints)
    try:
        prob.solve(solver=cp.GUROBI, verbose=False, reoptimize=True)
        
        if prob.status == "optimal":
            for i in range(3):
                ii, jj = vtx[i]
                u_fixed[ii, jj] = u_vars[i].value
                v_fixed[ii, jj] = v_vars[i].value
            # verify lambda is outside after fix
            u_vals_opt = np.array([u_vars[i].value for i in range(3)])
            v_vals_opt = np.array([v_vars[i].value for i in range(3)])
            lambda_fixed = compute_lambda_numpy(u_vals_opt, v_vals_opt)
            if not is_outside(lambda_fixed):
                lambda_before = compute_lambda_numpy(u_vals, v_vals)
                print(f"[Fallback] Triangle {vtx} optimized but lambda still inside")
                print(f"    lambda before fix: {lambda_before}")
                print(f"    lambda after  fix: {lambda_fixed}")
            return True
        if prob.status != "optimal":
            print(f"[Fallback] Triangle {vtx} status: {prob.status}, lambda_dec = {lambda_dec}")
            return False
    except Exception as e:
        print(f"[Fallback] Solver exception: {e}")
    return False

def patchwise_milp_fix_fallback(u_dec, v_dec, bitmap_ori, bitmap_dec, patch_size=50, epsilon=0.05, tau=1e-6, M=100.0):
    H, W = u_dec.shape  # (3600, 2400)
    u_fixed = u_dec.copy()
    v_fixed = v_dec.copy()

    total_patches = 0
    total_triangles_fixed = 0
    fallback_patches = 0
    fallback_triangles = 0
    failed_verification = 0

    # Iterate over patches
    for i in range(0, H - 1, patch_size):
        for j in range(0, W - 1, patch_size):
            print(f"Processing patch i={i}, j={j}")

            patch_u = {}
            patch_v = {}
            constraints = []
            objective_terms = []
            triangle_list = []

            i_end = min(i + patch_size, H - 1)
            j_end = min(j + patch_size, W - 1)

            # Loop through each cell in this patch
            for ii in range(i, i_end):
                for jj in range(j, j_end):
                    for tri_id in range(2):
                        # if not (bitmap_ori[ii, jj, tri_id] == 1 and bitmap_dec[ii, jj, tri_id] == 0):
                        if not (bitmap_ori[ii, jj, tri_id] == 0 and bitmap_dec[ii, jj, tri_id] == 1):
                            continue

                        if tri_id == 0:
                            vtx = [(ii, jj), (ii + 1, jj), (ii + 1, jj + 1)]
                        else:
                            vtx = [(ii, jj), (ii, jj + 1), (ii + 1, jj + 1)]
                        if triangle_has_zero_vertex(u_dec, v_dec, vtx):
                            continue 

                        triangle_list.append(vtx)
                        u_vars = [get_var(*v, patch_u, patch_v, u_dec, v_dec, constraints, objective_terms, epsilon)[0] for v in vtx]
                        v_vars = [get_var(*v, patch_u, patch_v, u_dec, v_dec, constraints, objective_terms, epsilon)[1] for v in vtx]
                        lambda_vec = compute_lambda_expr(u_vars, v_vars)

                        z_low = [cp.Variable(boolean=True) for _ in range(3)]
                        z_high = [cp.Variable(boolean=True) for _ in range(3)]
                        for k in range(3):
                            constraints += [
                                # lambda_vec[k] <= -tau + M * (1 - z_low[k]),
                                # lambda_vec[k] >= 1 + tau - M * (1 - z_high[k]),
                                lambda_vec[k] <= -0.001 + M * (1 - z_low[k]), #tau not too small
                                lambda_vec[k] >= 1 + 0.001 - M * (1 - z_high[k]),
                                z_low[k] + z_high[k] <= 1
                            ]
                        constraints.append(cp.sum(z_low + z_high) >= 1)

            if not constraints:
                continue

            total_patches += 1
            total_triangles_fixed += len(triangle_list)
            print(f"  -> Fixing {len(triangle_list)} triangles in patch ({i},{j})")

            prob = cp.Problem(cp.Minimize(cp.sum(objective_terms)), constraints)
            try:
                prob.solve(solver=cp.GUROBI, verbose=False, reoptimize=True)
            except Exception as e:
                print(f"  [Error] Solver failed in patch ({i},{j}): {str(e)}")
                continue

            if prob.status != "optimal":
                print(f"  [Warning] MILP not solved optimally at patch ({i},{j}), fallback to triangle-wise...")
                fallback_patches += 1
                for vtx in triangle_list:
                    success = fallback_single_triangle(u_dec, v_dec, u_fixed, v_fixed, vtx, epsilon, tau, M)
                    if success:
                        fallback_triangles += 1
                continue

            for (ii, jj), var in patch_u.items():
                u_fixed[ii, jj] = var.value
            for (ii, jj), var in patch_v.items():
                v_fixed[ii, jj] = var.value
                
            for vtx in triangle_list:
                u_vals = np.array([u_fixed[ii, jj] for (ii, jj) in vtx])
                v_vals = np.array([v_fixed[ii, jj] for (ii, jj) in vtx])
                lambda_fixed = compute_lambda_numpy(u_vals, v_vals)
                if not is_outside(lambda_fixed):
                    print(f"  [Verify Failed] Triangle {vtx} after patch MILP is still inside: {lambda_fixed}")
                    failed_verification += 1


    print(f"\n[Summary] Total patches processed: {total_patches}")
    print(f"[Summary] Total triangles fixed: {total_triangles_fixed}")
    print(f"[Summary] Fallback patches: {fallback_patches}")
    print(f"[Summary] Fallback triangles solved: {fallback_triangles}\n")
    print(f"[Summary] Patch-fixed but still invalid triangles: {failed_verification}")
    return u_fixed, v_fixed


In [38]:
u_fixed, v_fixed = patchwise_milp_fix_fallback(mesh_sz3_u, mesh_sz3_v, bitmap_ori, bitmap_sz3, patch_size=500, epsilon=0.2, tau=1e-4, M=100.0)

Processing patch i=0, j=0
  -> Fixing 1413 triangles in patch (0,0)
Processing patch i=0, j=500
  -> Fixing 917 triangles in patch (0,500)
Processing patch i=0, j=1000
  -> Fixing 1374 triangles in patch (0,1000)
Processing patch i=0, j=1500
  -> Fixing 1380 triangles in patch (0,1500)
  [Warning] MILP not solved optimally at patch (0,1500), fallback to triangle-wise...
Processing patch i=0, j=2000
  -> Fixing 942 triangles in patch (0,2000)
Processing patch i=0, j=2500
  -> Fixing 1192 triangles in patch (0,2500)
Processing patch i=0, j=3000
  -> Fixing 1092 triangles in patch (0,3000)
Processing patch i=0, j=3500
  -> Fixing 138 triangles in patch (0,3500)
Processing patch i=500, j=0
  -> Fixing 1911 triangles in patch (500,0)
Processing patch i=500, j=500
  -> Fixing 2981 triangles in patch (500,500)
Processing patch i=500, j=1000
  -> Fixing 2708 triangles in patch (500,1000)
Processing patch i=500, j=1500
  -> Fixing 1669 triangles in patch (500,1500)
Processing patch i=500, j=200

In [39]:
critical_cells = find_true_critical_cells(mesh_ori_u, mesh_ori_v)
print(f"检测到 {len(critical_cells)} 个包含 critical point 的三角形")
critical_cells_reconstructed = find_true_critical_cells(u_fixed, v_fixed)
print(f"检测到 {len(critical_cells_reconstructed)} 个包含 critical point 的三角形 in reconstructed mesh")

检测到 90826 个包含 critical point 的三角形
检测到 94394 个包含 critical point 的三角形 in reconstructed mesh


In [53]:
print(u_fixed.shape)

(2400, 3600)


In [30]:
diff = mesh_sz3_u != u_fixed
print("不同元素的总数:", np.sum(diff))

diff = mesh_sz3_v != v_fixed
print("不同元素的总数:", np.sum(diff))

equal = np.allclose(mesh_sz3_u, u_fixed, atol=0.2)
print("sz3 vs reconstructed 是否近似一致（容忍误差）:", equal)

equal = np.allclose(mesh_sz3_v, v_fixed, atol=0.2)
print("sz3 vs reconstructed 是否近似一致（容忍误差）:", equal)

equal = np.allclose( mesh_ori_u, u_fixed,atol=0.4)
print("original vs reconstructed 是否近似一致（容忍误差）:", equal)
equal = np.allclose( mesh_ori_v, v_fixed,atol=0.4)
print("original vs reconstructed 是否近似一致（容忍误差）:", equal)


不同元素的总数: 10440
不同元素的总数: 9929
sz3 vs reconstructed 是否近似一致（容忍误差）: True
sz3 vs reconstructed 是否近似一致（容忍误差）: True
original vs reconstructed 是否近似一致（容忍误差）: True
original vs reconstructed 是否近似一致（容忍误差）: True


In [ ]:
def compute_lambda(u, v):
    A = np.array([[2, 1], [1, 2]])
    b = np.array([u[0] - u[1], v[0] - v[1]])
    x, y = np.linalg.solve(A, b)
    return np.array([1 - x - y, x, y])

def is_inside(lambda_vec, tol=1e-4):
    return (
        np.all(lambda_vec >= -tol) and
        np.all(lambda_vec <= 1 + tol) and
        np.isclose(np.sum(lambda_vec), 1.0, atol=tol)
    )

def verify_bitmap_match(u_fixed, v_fixed, bitmap_ori, verbose=False):
    H, W = bitmap_ori.shape[:2]
    bitmap_eval = np.zeros_like(bitmap_ori, dtype=int)
    mismatches = 0

    for i in range(H):
        for j in range(W):
            for tri_id in range(2):
                if tri_id == 0:
                    vtx = [(i, j), (i + 1, j), (i + 1, j + 1)]
                else:
                    vtx = [(i, j), (i, j + 1), (i + 1, j + 1)]

                u_vals = np.array([u_fixed[ii, jj] for (ii, jj) in vtx])
                v_vals = np.array([v_fixed[ii, jj] for (ii, jj) in vtx])
                lamb = compute_lambda(u_vals, v_vals)
                bitmap_eval[i, j, tri_id] = int(is_inside(lamb))

                if bitmap_ori[i, j, tri_id] != bitmap_eval[i, j, tri_id]:
                    mismatches += 1
                    if verbose:
                        print(f"Mismatch at ({i}, {j}, {tri_id}): ori={bitmap_ori[i,j,tri_id]}, eval={bitmap_eval[i,j,tri_id]}")

    total = bitmap_ori.size
    matched = total - mismatches
    print(f"\n[Verification] Matched: {matched}/{total} ({matched / total:.4%})")
    print(f"[Verification] Mismatches: {mismatches}")

    return bitmap_eval

bitmap_eval = verify_bitmap_match(u_fixed, v_fixed, bitmap_ori, verbose=True)

NameError: name 'u_fixed' is not defined